In [1]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from keras.models import Sequential
import numpy as np
import os
from tqdm import tqdm
import pickle

In [2]:
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))  ### input shape
model.trainable = False

94765736/94765736 [==============================] - 53s 1us/step


In [3]:
model = Sequential([
    model,
    GlobalMaxPooling2D()
])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_max_pooling2d (Glob  (None, 2048)              0         
 alMaxPooling2D)                                                 
                                                                 
Total params: 23587712 (89.98 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [5]:
from numpy.linalg import norm

In [6]:
### function for doing below oprations
def feature_extraction(image_path, model):
  img = image.load_img(image_path, target_size=(224,224))     ### loading image form the path
  img_array = image.array_to_img(img)       ### converting it into array
  expanded_img_array = np.expand_dims(img_array, axis=0)      ### just expand as keras expect the batch of images
  preprocessed_img = preprocess_input(expanded_img_array)     ### so that all images should be in the format of the same as in imagenet
  result = model.predict(preprocessed_img).flatten()          ### just in 1d array
  normalized_result = result / norm(result)             ### normalizing for the zero centered

  return normalized_result


In [7]:
import os
filenames = []
for file in os.listdir("D:\#DATA Science\Deep Learning\Deep Learning Practice\Fashion Recomender System\images"):
  filenames.append(os.path.join("D:\#DATA Science\Deep Learning\Deep Learning Practice\Fashion Recomender System\images", file))

print(len(filenames))

44441


In [ ]:
import tqdm  # Make sure you've imported the tqdm module

feature_list = []

for file in tqdm.tqdm(filenames):  # Use tqdm.tqdm to iterate with a progress bar
    features = feature_extraction(file, model)
    feature_list.append(features)


In [9]:
import pickle
pickle.dump(feature_list,open('embeddings.pkl', 'wb'))
pickle.dump(filenames,open('filenames.pkl', 'wb'))